In [ ]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import warnings
from cvxopt import matrix, solvers
from KLIEP_importance_estimation import KLIEP_importance_estimation
import United_function_tools as uft
%matplotlib inline

## 1. Kernel ridge regression (KRR)

In [ ]:
def f_0(x):
    """define the mean regression function for 1-dimensional KRR (Example S1 in supplementary material)"""
    return np.exp(-1/(x**(2)))

def f_1(x):
    """define the mean regression function for 3-dimensional KRR (Example S2 in supplementary material)"""
    return np.sin(2*np.pi*x[:,0])-np.exp(-x[:,1]**2-x[:,2]**2)

### 1.1 1-dimensional case

In [ ]:
#Vary the regularization parameter lambda (bounded case)
mu_1=0 #the mean of source
mu_2=0.8 #the mean of target
sigma_1_sq=0.5  #the variance of source
sigma_2_sq=0.3   #the variance of target

n=500  #the size of training data
m=1000 #the size of testing data
t=int(m/2)  #the size of validation data
times=100 
lamb=10**np.linspace(-8,-2,11)
p=lamb.shape[0]

seed_tr=[i+1000 for i in range(times)] # the seed for random training data
seed_te=[i+2000 for i in range(times)] # the seed for random test data
seed_va=[i+3000 for i in range(times)] # the seed for random validation data

"""The array named all_MSE records the MSE  for the unweighted estimator;
  the array named all_MSE_W records the MSE  for the TIRW estimator with true weight;
  the array named all_MSE_What records the MSE  for the TIRW estimator with estimated weight."""
all_MSE=np.zeros([p,times]) 
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.sort(np.random.normal(mu_1,np.sqrt(sigma_1_sq),n))
    np.random.seed(seed_tr[k])
    y_train=f_0(x_train)+np.random.normal(0,0.05,n)
        
    np.random.seed(seed_te[k])
    x_test=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),m))
    np.random.seed(seed_te[k])
    y_test=f_0(x_test)+np.random.normal(0,0.05,m)
    
    np.random.seed(seed_va[k])
    x_valid=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),t))
    np.random.seed(seed_va[k])
    y_valid=f_0(x_valid)+np.random.normal(0,0.05,t)
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train.reshape(n,1), x_test.reshape(m,1))
    weight_hat=KLIEP.predict(x_train.reshape(n,1))
    # calculate the true weight
    weight=st.multivariate_normal.pdf(x_train,mu_2,sigma_2_sq)/st.multivariate_normal.pdf(x_train,mu_1,sigma_1_sq)

    #calculate the true conditional mean
    f_test=f_0(x_test)
    f_valid=f_0(x_valid)

    for i in range(p):
        
        predict_test=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,lam=lamb[i],sigma=1)
        predict_test_W=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                                          weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb[i],sigma=1)
        predict_test_What=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                                             weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb[i],sigma=1)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

In [ ]:
#Vary the source sample size n (bounded case)
n_list=np.array([200,400,800,1500,2000,3000])
l=n_list.shape[0]
m=1000     #the size of testing data
t=int(m/2) #the size of validation data

all_MSE=np.zeros([l,times])
all_MSE_W=np.zeros([l,times])
all_MSE_What=np.zeros([l,times])

for i in range(l):
    n=n_list[i]
    # for each n, we running 100 times
    seed_tr=[i+100000+n for i in range(times)] # the seed for random training data with size n
    seed_te=[i+200000+n for i in range(times)] # the seed for random test data
    seed_va=[i+300000+n for i in range(times)] # the seed for random validation data
    for k in range(times):
        np.random.seed(seed_tr[k])
        x_train=np.sort(np.random.normal(mu_1,np.sqrt(sigma_1_sq),n))
        np.random.seed(seed_tr[k])
        y_train=f_0(x_train)+np.random.normal(0,0.05,n)
        
        np.random.seed(seed_te[k])
        x_test=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),m))
        np.random.seed(seed_te[k])
        y_test=f_0(x_test)+np.random.normal(0,0.05,m)
    
        np.random.seed(seed_va[k])
        x_valid=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),t))
        np.random.seed(seed_va[k])
        y_valid=f_0(x_valid)+np.random.normal(0,0.05,t)
    
        #estimate the weight
        KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
        KLIEP.fit(x_train.reshape(n,1), x_test.reshape(m,1))
        weight_hat=KLIEP.predict(x_train.reshape(n,1))
        # calculate the true weight
        weight=st.multivariate_normal.pdf(x_train,mu_2,sigma_2_sq)/st.multivariate_normal.pdf(x_train,mu_1,sigma_1_sq)

        #calculate the true conditional mean
        f_test=f_0(x_test)
        f_valid=f_0(x_valid)
    
        lamb=10**(-4)
        predict_test=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,lam=lamb,sigma=1)
        predict_test_W=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                          weighted=True,weight=weight, truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb,sigma=1)
        predict_test_What=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                             weighted=True,weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb,sigma=1)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

In [ ]:
#Vary the target sample size m (bounded case)
m_list=np.array([500,1000,1500,2000,2500,3500])
l=m_list.shape[0]
n=500 #the size of training data

all_MSE=np.zeros([l,times])
all_MSE_W=np.zeros([l,times])
all_MSE_What=np.zeros([l,times])

for i in range(l):
    m=m_list[i]
    t=int(m/2) #the size of validation data
    # for each m, we running 100 times
    seed_tr=[i+110000+m for i in range(times)] # the seed for random training data with size n
    seed_te=[i+210000+m for i in range(times)] # the seed for random testing data
    seed_va=[i+310000+m for i in range(times)] # the seed for random validation data
    for k in range(times):
        np.random.seed(seed_tr[k])
        x_train=np.sort(np.random.normal(mu_1,np.sqrt(sigma_1_sq),n))
        np.random.seed(seed_tr[k])
        y_train=f_0(x_train)+np.random.normal(0,0.05,n)
        
        np.random.seed(seed_te[k])
        x_test=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),m))
        np.random.seed(seed_te[k])
        y_test=f_0(x_test)+np.random.normal(0,0.05,m)
    
        np.random.seed(seed_va[k])
        x_valid=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),t))
        np.random.seed(seed_va[k])
        y_valid=f_0(x_valid)+np.random.normal(0,0.05,t)

       #estimate the weight
        if m<=1000:
            num_param=0.1
        elif m<=2000:
            num_param=0.05
        else:
            num_param=0.015
        KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[num_param],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
        KLIEP.fit(x_train.reshape(n,1), x_test.reshape(m,1))
        weight_hat=KLIEP.predict(x_train.reshape(n,1))
        # calculate the true weight
        weight=st.multivariate_normal.pdf(x_train,mu_2,sigma_2_sq)/st.multivariate_normal.pdf(x_train,mu_1,sigma_1_sq)

        #calculate the true conditional mean
        f_test=f_0(x_test)
        f_valid=f_0(x_valid)
    
        lamb=10**(-4)
        predict_test=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,lam=lamb,sigma=1)
        predict_test_W=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                          weighted=True,weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb,sigma=1)
        predict_test_What=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                             weighted=True,weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb,sigma=1)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

In [ ]:
#Vary the regularization parameter lambda (unbounded case)
mu_1=0 #the mean of P
mu_2=1.5 #the mean of Q
sigma_1_sq=0.3  #the variance of P
sigma_2_sq=0.5   #the variance of Q

n=500     #the size of training data
m=1000    #the size of testing data
t=int(m/2)  #the size of validation data
times=100
lamb=10**np.linspace(-7,-2,9)
p=lamb.shape[0]

seed_tr=[i+1000 for i in range(times)] # the seed for random training data
seed_te=[i+2000 for i in range(times)] # the seed for random testing data
seed_va=[i+3000 for i in range(times)] # the seed for random validation data

all_MSE=np.zeros([p,times])
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])
for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.sort(np.random.normal(mu_1,np.sqrt(sigma_1_sq),n))
    np.random.seed(seed_tr[k])
    y_train=f_0(x_train)+np.random.normal(0,0.05,n)
        
    np.random.seed(seed_te[k])
    x_test=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),m))
    np.random.seed(seed_te[k])
    y_test=f_0(x_test)+np.random.normal(0,0.05,m)
    
    np.random.seed(seed_va[k])
    x_valid=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),t))
    np.random.seed(seed_va[k])
    y_valid=f_0(x_valid)+np.random.normal(0,0.05,t)
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train.reshape(n,1), x_test.reshape(m,1))
    weight_hat=KLIEP.predict(x_train.reshape(n,1))
    # calculate the true weight
    weight=st.multivariate_normal.pdf(x_train,mu_2,sigma_2_sq)/st.multivariate_normal.pdf(x_train,mu_1,sigma_1_sq)
    
    #calculate the true conditional mean
    f_test=f_0(x_test)
    f_valid=f_0(x_valid)

    for i in range(p):
        
        predict_test=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,lam=lamb[i],sigma=1)
        predict_test_W=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                          weighted=True,weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),lam=lamb[i],sigma=1)
        predict_test_What=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                weighted=True,weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),lam=lamb[i],sigma=1)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

### 1.2 3-dimensional  case

In [ ]:
#Vary the regularization parameter lambda (bounded case)
sigma=0.3
alpha=3
beta=4
alpha_pr=2.5
beta_pr=1.5

n=500       #the size of training data
m=1000      #the size of testing data
t=int(m/2)  #the size of validation data

times=100 
lamb=10**np.linspace(-9,-3,13)
p=lamb.shape[0]

seed_tr=[i+1000 for i in range(times)] # the seed for random training data
seed_te=[i+2000 for i in range(times)] # the seed for random testing data
seed_va=[i+3000 for i in range(times)] # the seed for random validation data

all_MSE=np.zeros([p,times])
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.random.uniform(0,1,size=(n,3))
    x_train[:,0]=np.random.beta(alpha_pr,beta_pr,n)
    np.random.seed(seed_tr[k])
    y_train=f_1(x_train)+sigma*st.norm(0,1).rvs(n)
        
    np.random.seed(seed_te[k])
    x_test=np.random.uniform(0,1,size=(m,3))
    x_test[:,0]=np.random.beta(alpha,beta,m)
    np.random.seed(seed_te[k])
    y_test=f_1(x_test)+sigma*st.norm(0,1).rvs(m)
    
    np.random.seed(seed_va[k])
    x_valid=np.random.uniform(0,1,size=(t,3))
    x_valid[:,0]=np.random.beta(alpha,beta,t)
    np.random.seed(seed_va[k])
    y_valid=f_1(x_valid)+sigma*st.norm(0,1).rvs(t)
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train,x_test)
    weight_hat=KLIEP.predict(x_train)
    #calculate the conditional mean
    weight=st.beta(alpha,beta).pdf(x_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(x_train[:,0])
    
    #calculate the true conditional mean
    f_test=f_1(x_test)
    f_valid=f_1(x_valid)
    
    for i in range(p):
        
        predict_test=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,lam=lamb[i],sigma=1)
        predict_test_W=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                    weighted=True,weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb[i],sigma=1)
        predict_test_What=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                    weighted=True,weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),lam=lamb[i],sigma=1)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

In [ ]:
#Vary the regularization parameter lambda (unbounded case)
sigma=0.3
alpha=3
beta=6
alpha_pr=4
beta_pr=1

n=500       #the size of training data
m=1000      #the size of testing data
t=int(m/2)  #the size of validation data

times=100 
lamb=10**np.linspace(-8,-2.5,11)
p=lamb.shape[0]

seed_tr=[i+1000 for i in range(times)] # the seed for random training data
seed_te=[i+2000 for i in range(times)] # the seed for random testing data
seed_va=[i+3000 for i in range(times)] # the seed for random validation data

all_MSE=np.zeros([p,times])
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.random.uniform(0,1,size=(n,3))
    x_train[:,0]=np.random.beta(alpha_pr,beta_pr,n)
    np.random.seed(seed_tr[k])
    y_train=f_1(x_train)+sigma*st.norm(0,1).rvs(n)
        
    np.random.seed(seed_te[k])
    x_test=np.random.uniform(0,1,size=(m,3))
    x_test[:,0]=np.random.beta(alpha,beta,m)
    np.random.seed(seed_te[k])
    y_test=f_1(x_test)+sigma*st.norm(0,1).rvs(m)
    
    np.random.seed(seed_va[k])
    x_valid=np.random.uniform(0,1,size=(t,3))
    x_valid[:,0]=np.random.beta(alpha,beta,t)
    np.random.seed(seed_va[k])
    y_valid=f_1(x_valid)+sigma*st.norm(0,1).rvs(t)
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train,x_test)
    weight_hat=KLIEP.predict(x_train)
    # calculate the true weight
    weight=st.beta(alpha,beta).pdf(x_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(x_train[:,0])
    
    #calculate the true conditional mean
    f_test=f_1(x_test)
    f_valid=f_1(x_valid)
    for i in range(p): 
        predict_test=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,lam=lamb[i],sigma=1)
        predict_test_W=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                    weighted=True,weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),lam=lamb[i],sigma=1)
        predict_test_What=uft.KRR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                                    weighted=True,weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),lam=lamb[i],sigma=1)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[:,k]=mean_squared_error(f_test, predict_test_What[1])

## 2. Kernel quantile regression (KQR)

In [ ]:
def f_3(x):
    """define the true conditional quantile function for 1-dimensional KQR (Example S3 in supplementary material)"""
    return np.sin(np.pi*x)
def regression_1(x,sigma,tau=0.5,r=1):
    """define the regression function for 1-dimensional KQR (Example S3 in supplementary material)"""
    return f_3(x)-(1+r*(x-0.5)**2)*sigma*st.norm.ppf(tau,0,1)

def f_4(x):
    """define the true conditional quantile function for 3-dimensional KQR (Example S4 in supplementary material)"""
    return np.sin(1.5*np.pi*x[:,0])-np.exp(-x[:,1]**2-x[:,2]**2)

def regression_2(x,sigma,df=4,tau=0.5,r=1):
    """define the regression function for 1-dimensional KQR (Example S4 in supplementary material)"""
    return f_4(x)-(1+r*x[:,0])*sigma*st.t(df).ppf(tau)

### 2.1 1-dimensional  case

In [ ]:
#Vary the regularization parameter lambda (bounded case)
sigma=0.3
tau=0.5
r=1
mu_1=0 #the mean of P
mu_2=0.5 #the mean of Q
sigma_1_sq=0.4  #the variance of P
sigma_2_sq=0.3   #the variance of Q

n=500 #the size of training data
m=1000 #the size of testing data
t=int(m/2)  #the size of validation data

times=100

lamb=10**np.linspace(-6,-1,11)
p=lamb.shape[0]
C=1/(n*lamb)


seed_tr=[i+1000 for i in range(times)] # the seed for random training data
seed_te=[i+2000 for i in range(times)] # the seed for random testing data
seed_va=[i+3000 for i in range(times)] # the seed for random validation data

p=lamb.shape[0]

all_MSE=np.zeros([p,times])
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])
all_loss=np.zeros([p,times])
all_loss_W=np.zeros([p,times])
all_loss_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.sort(np.random.normal(mu_1,np.sqrt(sigma_1_sq),n))
    np.random.seed(seed_tr[k])
    y_train=regression_1(x_train,sigma=sigma,tau=tau,r=r)+(1+r*(x_train-0.5)**2)*sigma*np.random.normal(0,1,n)
        
    np.random.seed(seed_te[k])
    x_test=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),m))
    np.random.seed(seed_te[k])
    y_test=regression_1(x_test,sigma=sigma,tau=tau,r=r)+(1+r*(x_test-0.5)**2)*sigma*np.random.normal(0,1,m)
    
    np.random.seed(seed_va[k])
    x_valid=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),t))
    np.random.seed(seed_va[k])
    y_valid=regression_1(x_valid,sigma=sigma,tau=tau,r=r)+(1+r*(x_valid-0.5)**2)*sigma*np.random.normal(0,1,t)
    
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train.reshape(n,1), x_test.reshape(m,1))
    weight_hat=KLIEP.predict(x_train.reshape(n,1))
    # calculate the true weight
    weight=st.multivariate_normal.pdf(x_train,mu_2,sigma_2_sq)/st.multivariate_normal.pdf(x_train,mu_1,sigma_1_sq)
    #calculate the true quantile
    f_test=f_3(x_test)
    f_valid=f_3(x_valid)
    for i in range(p):
        
        predict_test=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,C=C[i],sigma=1,tau=tau)
        predict_test_W=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,
                        weighted=True,weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),tau=tau,C=C[i],sigma=1,loss='MSE')
        predict_test_What=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                          weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,2.5,30)),tau=tau,C=C[i],sigma=1,loss='MSE')
    
        all_loss[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test[1],f_true=f_test,tau=tau)
        all_loss_W[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_W[1],f_true=f_test,tau=tau)
        all_loss_What[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_What[1],f_true=f_test,tau=tau)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

In [ ]:
#Vary the regularization parameter lambda (unbounded case)
sigma=0.3
tau=0.5
r=1
mu_1=0 #the mean of P
mu_2=1 #the mean of Q
sigma_1_sq=0.3  #the variance of P
sigma_2_sq=0.5   #the variance of Q

n=500 #the size of training data
m=1000 #the size of testing data
t=int(m/2)  #the size of validation data

times=100

lamb=10**np.linspace(-6,-1,11)
p=lamb.shape[0]
C=1/(n*lamb)

seed_tr=[i+1000 for i in range(times)] # the seed for random training data
seed_te=[i+2000 for i in range(times)] # the seed for random testing data
seed_va=[i+3000 for i in range(times)] # the seed for random validation data

all_MSE=np.zeros([p,times])
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])
all_loss=np.zeros([p,times])
all_loss_W=np.zeros([p,times])
all_loss_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.sort(np.random.normal(mu_1,np.sqrt(sigma_1_sq),n))
    np.random.seed(seed_tr[k])
    y_train=regression_1(x_train,sigma=sigma,tau=tau,r=r)+(1+r*(x_train-0.5)**2)*sigma*np.random.normal(0,1,n)
        
    np.random.seed(seed_te[k])
    x_test=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),m))
    np.random.seed(seed_te[k])
    y_test=regression_1(x_test,sigma=sigma,tau=tau,r=r)+(1+r*(x_test-0.5)**2)*sigma*np.random.normal(0,1,m)
    
    np.random.seed(seed_va[k])
    x_valid=np.sort(np.random.normal(mu_2,np.sqrt(sigma_2_sq),t))
    np.random.seed(seed_va[k])
    y_valid=regression_1(x_valid,sigma=sigma,tau=tau,r=r)+(1+r*(x_valid-0.5)**2)*sigma*np.random.normal(0,1,t)
    
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train.reshape(n,1), x_test.reshape(m,1))
    weight_hat=KLIEP.predict(x_train.reshape(n,1))
    # calculate the true weight
    weight=st.multivariate_normal.pdf(x_train,mu_2,sigma_2_sq)/st.multivariate_normal.pdf(x_train,mu_1,sigma_1_sq)

    #calculate the true quantile
    f_test=f_3(x_test)
    f_valid=f_3(x_valid)
    
    for i in range(p):
        
        predict_test=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,C=C[i],sigma=1,tau=tau)
        predict_test_W=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                          weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),tau=tau,C=C[i],sigma=1,loss='MSE')
        predict_test_What=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                          weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),tau=tau,C=C[i],sigma=1,loss='MSE')
    
        all_loss[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test[1],f_true=f_test,tau=tau)
        all_loss_W[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_W[1],f_true=f_test,tau=tau)
        all_loss_What[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_What[1],f_true=f_test,tau=tau)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

### 2.2 3-dimensional  case

In [ ]:
#Vary the regularization parameter lambda (bounded case)
sigma=0.3
tau=0.5
df=4
r=1
alpha=3
beta=6
alpha_pr=2.5
beta_pr=1.5
times=100
n=500 #the size of training data
m=1000 #the size of testing data
t=int(m/2)  #the size of validation data

lamb=10**np.linspace(-6.5,-1,11)
p=lamb.shape[0]
C=1/(n*lamb)

seed_tr=[i+100 for i in range(times)] # the seed for random training data
seed_te=[i+200 for i in range(times)] # the seed for random testing data
seed_va=[i+300 for i in range(times)] # the seed for random validation data



all_MSE=np.zeros([p,times])
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])
all_loss=np.zeros([p,times])
all_loss_W=np.zeros([p,times])
all_loss_What=np.zeros([p,times])

for k in range(10):
    np.random.seed(seed_tr[k])
    x_train=np.random.uniform(0,1,size=(n,3))
    x_train[:,0]=np.random.beta(alpha_pr,beta_pr,n)
    np.random.seed(seed_tr[k])
    y_train=regression_2(x_train,sigma=sigma,tau=tau,df=df,r=r)+(1+r*x_train[:,0])*sigma*st.t(df).rvs(n)
        
    np.random.seed(seed_te[k])
    x_test=np.random.uniform(0,1,size=(m,3))
    x_test[:,0]=np.random.beta(alpha,beta,m)
    np.random.seed(seed_te[k])
    y_test=regression_2(x_test,sigma=sigma,tau=tau,df=df,r=r)+(1+r*x_test[:,0])*sigma*st.t(df).rvs(m)
    
    np.random.seed(seed_va[k])
    x_valid=np.random.uniform(0,1,size=(t,3))
    x_valid[:,0]=np.random.beta(alpha,beta,t)
    np.random.seed(seed_va[k])
    y_valid=regression_2(x_valid,sigma=sigma,tau=tau,df=df,r=r)+(1+r*x_valid[:,0])*sigma*st.t(df).rvs(t)
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train,x_test)
    weight_hat=KLIEP.predict(x_train)
    # calculate the true weight
    weight=st.beta(alpha,beta).pdf(x_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(x_train[:,0])

    #calculate the true quantile
    f_test=f_4(x_test)
    f_valid=f_4(x_valid)
    
    for i in range(p):
        
        predict_test=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,C=C[i],sigma=1,tau=tau)
        predict_test_W=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                          weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,2,40)),tau=tau,C=C[i],sigma=1,loss='MSE')
        predict_test_What=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                          weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,2,40)),tau=tau,C=C[i],sigma=1,loss='MSE')

        all_loss[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test[1],f_true=f_test,tau=tau)
        all_loss_W[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_W[1],f_true=f_test,tau=tau)
        all_loss_What[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_What[1],f_true=f_test,tau=tau)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

In [ ]:
#Vary the regularization parameter lambda (unbounded case)
sigma=0.3
tau=0.5
df=4
r=1
alpha=3
beta=6
alpha_pr=5.5
beta_pr=1.5
times=100
n=500 #the size of training data
m=1000 #the size of testing data
t=int(m/2)  #the size of validation data

lamb_log=np.linspace(-6.5,-1,11)
lamb=10**lamb_log
tC=1/(n*lamb)

seed_tr=[i+1000 for i in range(times)] # the seed for random training data
seed_te=[i+2000 for i in range(times)] # the seed for random testing data
seed_va=[i+3000 for i in range(times)] # the seed for random validation data

p=lamb.shape[0]

all_MSE=np.zeros([p,times])
all_MSE_W=np.zeros([p,times])
all_MSE_What=np.zeros([p,times])
all_loss=np.zeros([p,times])
all_loss_W=np.zeros([p,times])
all_loss_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.random.uniform(0,1,size=(n,3))
    x_train[:,0]=np.random.beta(alpha_pr,beta_pr,n)
    np.random.seed(seed_tr[k])
    y_train=regression_2(x_train,sigma=sigma,tau=tau,df=df,r=r)+(1+r*x_train[:,0])*sigma*st.t(df).rvs(n)
        
    np.random.seed(seed_te[k])
    x_test=np.random.uniform(0,1,size=(m,3))
    x_test[:,0]=np.random.beta(alpha,beta,m)
    np.random.seed(seed_te[k])
    y_test=regression_2(x_test,sigma=sigma,tau=tau,df=df,r=r)+(1+r*x_test[:,0])*sigma*st.t(df).rvs(m)
    
    np.random.seed(seed_va[k])
    x_valid=np.random.uniform(0,1,size=(t,3))
    x_valid[:,0]=np.random.beta(alpha,beta,t)
    np.random.seed(seed_va[k])
    y_valid=regression_2(x_valid,sigma=sigma,tau=tau,df=df,r=r)+(1+r*x_valid[:,0])*sigma*st.t(df).rvs(t)
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train,x_test)
    weight_hat=KLIEP.predict(x_train)
    # calculate the true weight
    weight=st.beta(alpha,beta).pdf(x_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(x_train[:,0])

    #calculate the true quantile
    f_test=f_4(x_test)
    f_valid=f_4(x_valid)
    
    for i in range(p):
        
        predict_test=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,C=C[i],sigma=1,tau=tau)
        predict_test_W=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                          weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),tau=tau,C=C[i],sigma=1,loss='MSE')
        predict_test_What=uft.KQR_estimation(x=x_test,x_train=x_train,y_train=y_train,x_valid=x_valid,f_valid=f_valid,weighted=True,
                          weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,4,40)),tau=tau,C=C[i],sigma=1,loss='MSE')

        all_loss[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test[1],f_true=f_test,tau=tau)
        all_loss_W[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_W[1],f_true=f_test,tau=tau)
        all_loss_What[i,k]=uft.empirical_KQR(y=y_test,f_hat= predict_test_What[1],f_true=f_test,tau=tau)
        all_MSE[i,k]=mean_squared_error(f_test, predict_test[1])
        all_MSE_W[i,k]=mean_squared_error(f_test, predict_test_W[1])
        all_MSE_What[i,k]=mean_squared_error(f_test, predict_test_What[1])

## 3.Kernel logistic  regression (KLR)

In [ ]:
def f_5(x):
    """define the mean regression function for 3-dimensional KQR (Example S5 in supplementary material)"""
    return 3*np.sin(3*np.pi*x[:,0])-x[:,0]**2+np.exp(x[:,1]**2-x[:,2]**2)

### 3.1 3-dimensional  case

In [ ]:
#Vary the regularization parameter lambda (bounded case)
alpha=3
beta=4
alpha_pr=2.5
beta_pr=2

n=500   #the size of training data
m=1000  #the size of testing data
t=int(m/2)  #the size of validation data
T=10 #iterations 
times=100

lamb=10**np.linspace(-10,-1,7)

seed_tr=[i+100 for i in range(times)] # the seed for random training data
seed_te=[i+200 for i in range(times)] # the seed for random testing data
seed_va=[i+300 for i in range(times)] # the seed for random validation data

p=lamb.shape[0]
"""The array named accuracy records the accuracy rate  for the unweighted estimator;
   the array named accuracy_W records the accuracy rate  for the TIRW estimator with true weight;
  the array named accuracy_What records the accuracy rate  for the TIRW estimator with estimated weight;
  the array named loss records empirical excess risk for the unweighted estimator;
  the array named loss_W records empirical excess risk for the TIRW estimator with true weight;
  the array named loss_What records empirical excess risk for the TIRW estimator with estimated weight;"""
accuracy=np.zeros([p,times])
accuracy_W=np.zeros([p,times])
accuracy_What=np.zeros([p,times])
loss=np.zeros([p,times])
loss_W=np.zeros([p,times])
loss_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.random.uniform(0,1,size=(n,3))
    x_train[:,0]=np.random.beta(alpha_pr,beta_pr,n)
    np.random.seed(seed_tr[k])
    prob_tr=1/(1+np.exp(-f_5(x_train)))
    y_train=np.random.binomial(1,p=prob_tr)
    y_train[y_train==0]=-1
          
    np.random.seed(seed_te[k])
    x_test=np.random.uniform(0,1,size=(m,3))
    x_test[:,0]=np.random.beta(alpha,beta,m)
    prob_te=1/(1+np.exp(-f_5(x_test)))
    y_test=np.random.binomial(1,p=prob_te)
    y_test[y_test==0]=-1
    
    np.random.seed(seed_va[k])
    x_valid=np.random.uniform(0,1,size=(t,3))
    x_valid[:,0]=np.random.beta(alpha,beta,t)
    prob_va=1/(1+np.exp(-f_5(x_valid)))
    y_valid=np.random.binomial(1,p=prob_va)
    y_valid[y_valid==0]=-1
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train,x_test)
    weight_hat=KLIEP.predict(x_train)
    # calculate the true weight
    weight=st.beta(alpha,beta).pdf(x_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(x_train[:,0])
    for i in range(p):

        result_1=uft.KLR_estimation(x=x_test,x_train=x_train,y_train=y_train,T=T,weighted=False,lamb=lamb[i],sigma=1)
        accuracy[i,k]=(y_test==result_1[1]).sum()/y_test.shape[0]
        loss[i,k]=uft.empirical_KLR(y=y_test,f_hat=result_1[0],f_true=f_5(x_test))

        result_2=uft.KLR_estimation(x=x_test,x_train=x_train,y_train=y_train,T=T,x_valid=x_valid,y_valid=y_valid,weighted=True,
                             weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,1,20)),lamb=lamb[i],sigma=1)
        accuracy_W[i,k]=(y_test==result_2[1]).sum()/y_test.shape[0]
        loss_W[i,k]=uft.empirical_KLR(y=y_test,f_hat=result_2[0],f_true=f_5(x_test))
        
        result_3=uft.KLR_estimation(x=x_test,x_train=x_train,y_train=y_train,T=T,x_valid=x_valid,y_valid=y_valid,weighted=True,
                             weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,1,20)),lamb=lamb[i],sigma=1)
        accuracy_What[i,k]=(y_test==result_3[1]).sum()/y_test.shape[0]
        loss_What[i,k]=uft.empirical_KLR(y=y_test,f_hat=result_3[0],f_true=f_5(x_test))

In [ ]:
#Vary the regularization parameter lambda (unbounded case)
alpha=3
beta=6
alpha_pr=4
beta_pr=1


n=500   #the size of training data
m=1000  #the size of testing data
t=int(m/2)  #the size of validation data
T=10 #iterations 
times=100

lamb=10**np.linspace(-10,-1,7)

seed_tr=[i+100 for i in range(times)] # the seed for random training data
seed_te=[i+200 for i in range(times)] # the seed for random testing data
seed_va=[i+300 for i in range(times)] # the seed for random validation data

p=lamb.shape[0]
accuracy=np.zeros([p,times])
accuracy_W=np.zeros([p,times])
accuracy_What=np.zeros([p,times])
loss=np.zeros([p,times])
loss_W=np.zeros([p,times])
loss_What=np.zeros([p,times])

for k in range(times):
    np.random.seed(seed_tr[k])
    x_train=np.random.uniform(0,1,size=(n,3))
    x_train[:,0]=np.random.beta(alpha_pr,beta_pr,n)
    np.random.seed(seed_tr[k])
    prob_tr=1/(1+np.exp(-f_5(x_train)))
    y_train=np.random.binomial(1,p=prob_tr)
    y_train[y_train==0]=-1
          
    np.random.seed(seed_te[k])
    x_test=np.random.uniform(0,1,size=(m,3))
    x_test[:,0]=np.random.beta(alpha,beta,m)
    prob_te=1/(1+np.exp(-f_5(x_test)))
    y_test=np.random.binomial(1,p=prob_te)
    y_test[y_test==0]=-1
    
    np.random.seed(seed_va[k])
    x_valid=np.random.uniform(0,1,size=(t,3))
    x_valid[:,0]=np.random.beta(alpha,beta,t)
    prob_va=1/(1+np.exp(-f_5(x_valid)))
    y_valid=np.random.binomial(1,p=prob_va)
    y_valid[y_valid==0]=-1
    
    #estimate the weight
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=5,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train,x_test)
    weight_hat=KLIEP.predict(x_train)
    # calculate the true weight
    weight=st.beta(alpha,beta).pdf(x_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(x_train[:,0])
    for i in range(p):

        result_1=uft.KLR_estimation(x=x_test,x_train=x_train,y_train=y_train,T=T,weighted=False,lamb=lamb[i],sigma=1)
        accuracy[i,k]=(y_test==result_1[1]).sum()/y_test.shape[0]
        loss[i,k]=uft.empirical_KLR(y=y_test,f_hat=result_1[0],f_true=f_5(x_test))

        
        result_2=uft.KLR_estimation(x=x_test,x_train=x_train,y_train=y_train,T=T,x_valid=x_valid,y_valid=y_valid,weighted=True,
                             weight=weight,truncation=True,tr=np.exp(np.linspace(-4.5,4,30)),lamb=lamb[i],sigma=1)
        accuracy_W[i,k]=(y_test==result_2[1]).sum()/y_test.shape[0]
        loss_W[i,k]=uft.empirical_KLR(y=y_test,f_hat=result_2[0],f_true=f_5(x_test))
        
        result_3=uft.KLR_estimation(x=x_test,x_train=x_train,y_train=y_train,T=T,x_valid=x_valid,y_valid=y_valid,weighted=True,
                             weight=weight_hat,truncation=True,tr=np.exp(np.linspace(-4.5,4,30)),lamb=lamb[i],sigma=1)
        accuracy_What[i,k]=(y_test==result_3[1]).sum()/y_test.shape[0]
        loss_What[i,k]=uft.empirical_KLR(y=y_test,f_hat=result_3[0],f_true=f_5(x_test))

##  4.Kernel support vector machine (KSVM)

### 4.1 Raisin_Dataset

In [ ]:
data_path ="Raisin_Dataset.xlsx"
data=pd.read_excel(data_path)
data['Class'][data['Class']=='Kecimen']=1
data['Class'][data['Class']!=1]=-1

x=np.array(data.iloc[:,:7])
y=np.array(data.iloc[:,-1])

x=(x-x.mean(axis=0))/x.std(axis=0) #scale the data
n=y.shape[0]

y=y.astype(np.double)

le=6 #choose the shift level 

C=10**np.array([-2.,-1.,0,1.,2.,3.])
times=100
n=y.shape[0]
l=len(C)

seed_list=[i+100 for i in range(times)] # the seed for random training data

"""The array named unweight records the accuracy rate  for the unweighted estimator;
   the array named weight records the accuracy rate  for the IRW estimator;
   the array named weight_tr records the accuracy rate  for the TIRW estimator."""
unweight=np.zeros([l,times])
weight=np.zeros([l,times])
weight_tr=np.zeros([l,times])

p=(x[:,0]-x[:,0].min())**2/le
p=np.clip(p,a_max=1,a_min=0)

for i in range(times):
    seed=seed_list[i]
    #Splitting into training data and test data
    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    
    np.random.seed(seed)
    s=np.random.binomial(1,p,p.shape[0])
    for k in range(n):
        if s[k]==0:
            x_train.append(x[k,:])
            y_train.append(y[k])
        else:
            x_test.append(x[k,:])
            y_test.append(y[k])

    x_train=np.array(x_train)
    y_train=np.array(y_train)

    x_test=np.array(x_test)
    y_test=np.array(y_test)
    
    KLIEP=KLIEP_importance_estimation(max_iter=5000,num_params=[0.1],epsilon=0.0001,cv=3,sigmas=[0.25,0.5,0.6,0.7,0.8,1,2])
    KLIEP.fit(x_train,x_test)
    weight_hat=KLIEP.predict(x_train)
    tr=np.exp(np.linspace(min(0.5,np.log(weight_hat.max())),np.log(weight_hat.max()),10))
    for t in range(l):
        c=C[t]
        result=uft.KSVM_estimation(x_test,weighted=False,x_train=x_train,y_train=y_train,C=c,sigma=1)
        y_pre=result[1].copy()
        y_pre[y_pre>=0]=1
        y_pre[y_pre<0]=-1
        y_pre=y_pre.reshape(y_test.shape[0],)
        unweight[t,i]=(y_test==y_pre).sum()/y_test.shape[0]
        
        result=uft.KSVM_estimation(x_test,weighted=True,weight=weight_hat,x_train=x_train,y_train=y_train,C=c,sigma=1)
        y_pre=result[1].copy()
        y_pre[y_pre>=0]=1
        y_pre[y_pre<0]=-1
        y_pre=y_pre.reshape(y_test.shape[0],)
        weight[t,i]=(y_test==y_pre).sum()/y_test.shape[0]
        
        tr_op=uft.cross_validation(x_train,y_train,weight=weight_hat,tr=tr,cv_times=3,random_state=None, ty='one-zero',C=c)
        weight_=np.clip(weight_hat,a_max=tr_op,a_min=0)
        result=uft.KSVM_estimation(x_test,weighted=True,weight=weight_,x_train=x_train,y_train=y_train,C=c,sigma=1)
        y_pre=result[1].copy()
        y_pre[y_pre>=0]=1
        y_pre[y_pre<0]=-1
        y_pre=y_pre.reshape(y_test.shape[0],)
        weight_tr[t,i]=(y_test==y_pre).sum()/y_test.shape[0]